In [1]:
from selenium import webdriver
import time
import pandas as pd
from bs4 import BeautifulSoup

 
path = "C:\\workspace\\특화프로젝트\\chromedriver-win64\\chromedriver.exe" # 웹드라이버 실행
 
driver = webdriver.Chrome() # 드라이버 경로 설정
url_list = [] # url을 저장하기 위한 변수
content_list = []


# 페이지를 순회합니다.
for i in range(1, 11):
    url = f'https://bbs.chosun.com/messagelist/list.bbs?bbs_id=9999300015&current_page={i}'
    driver.get(url)
    time.sleep(1)  # 페이지 로딩을 위해 잠시 대기합니다

    soup = BeautifulSoup(driver.page_source, 'html.parser')
    # 게시글 목록 추출
    rows = soup.find_all('tr', {'height': '28'})

    for row in rows:
        try:
            title_url = "https://bbs.chosun.com"+row.find('td', {'class': 'list_txt02'}).find('a')['href']
            url_list.append(title_url)
        except Exception as e:
            print(f"Error finding element: {e}")

print("url 수집 끝")
driver.close()

print("수집한 url 개수 : "+str(len(url_list)))
# print(url_list)

url 수집 끝
수집한 url 개수 : 200


In [2]:
from bs4 import BeautifulSoup

driver = webdriver.Chrome()
df = pd.DataFrame(columns=['URL', 'title', 'content', 'question']) # content를 누적하기 위한 변수

for index, url in enumerate(url_list):
    title_text = ""
    content_text = ""
    question_text = ""
    driver.get(url)
    time.sleep(1)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    row = soup.find('tr', {'height': '29px', 'class': 'ivory_titleBg'})
    title_tag = row.find('td', {'colspan': '7', 'class': 'list_txt05'}).find('b')
    if title_tag:
        title_text = title_tag.text.strip()
    else:
        title_text = "No Title Found"
        
    # 내용 추출
    content_div = soup.find('div', {'id': 'blogContents'})
    if content_div:
        spans = content_div.find_all('span', style=lambda value: value and 'font-size: 14pt' in value and 'line-height:1.8' in value)
    
        # span 태그의 텍스트를 추출합니다.
        content_text = "\n".join(span.get_text(separator='\n', strip=True) for span in spans)
    
    # 질문 추출
    question_divs = soup.find_all('p', style=lambda value: value and 'background:#f5efe5;' in value)
    if question_divs:
        questions = [q.get_text(strip=True) for q in question_divs]
        question_text = '\n'.join(questions)
        
    content_list.append({
        'URL': url,
        'title': title_text,
        'content': content_text,
        'question': question_text
    })
    
    if (index + 1) % 10 == 0:
        print(f"Processed {index + 1} URLs")

df = pd.DataFrame(content_list)
# print(df)

driver.quit()

Processed 10 URLs
Processed 20 URLs
Processed 30 URLs
Processed 40 URLs
Processed 50 URLs
Processed 60 URLs
Processed 70 URLs
Processed 80 URLs
Processed 90 URLs
Processed 100 URLs
Processed 110 URLs
Processed 120 URLs
Processed 130 URLs
Processed 140 URLs
Processed 150 URLs
Processed 160 URLs
Processed 170 URLs
Processed 180 URLs
Processed 190 URLs
Processed 200 URLs


In [3]:
file_path = "../data/raw/news/nie_chosun_crawled.csv"
df.to_csv(file_path, index=False, encoding='utf-8-sig')

print("Data saved")

Data saved
